In [1]:
%store -z # remove all variables from variable storage

In [2]:
# load credentials from environment variables
%load_ext dotenv
%dotenv

# util
import numpy as np
import boto3
import json

# date & time
import time
from datetime import timezone, date, datetime
from dateutil.relativedelta import relativedelta as rdelta
from dateutil.rrule import rrule, MONTHLY

# Oauth
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

## Get authorization token

In [4]:
# Your client credentials
client_id = %env SH_CLIENT_ID
client_secret = %env SH_CLIENT_SECRET

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id, client_secret=client_secret)

resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")

## Configure request (evalscript)

Enter start and end date, input bands, indices. The resulting files will have two time intervals per month, being split at `day_of_new_interval`.

In [4]:
startdate = date(2018,7,1) # Y,M,D
enddate = date(2018,9,15)  # Y,M,D

input_bands = ["B03",
               "B04",
               "B05",
               "B08"]
indices = ['NDVI',
           "NDWI",
           "CVI"]

bucket_name = "eox-masterdatacube"

day_of_new_interval = 16 # leave this unchanged in most of the cases

### Calculate parameters

In [6]:
starttime = datetime(*startdate.timetuple()[:6])
endtime = datetime(*enddate.timetuple()[:6])

d=day_of_new_interval
dates = list(rrule(MONTHLY, dtstart=startdate, until=enddate, bymonthday=[1,d-1,d,31]))
dates = [starttime] + dates if dates[0] != starttime else dates
dates = dates + [endtime] if dates[-1] != endtime else dates

starts = dates[0::2]
starts = [int(d.timestamp()) for d in starts] # timestamps for arithmetic
ends   = [d+rdelta(hour=23, minute=59, second=59) for d in dates[1::2]]
ends   = [int(d.timestamp()) for d in ends]   # timestamps for arithmetic
avg_times = list(np.mean(list(zip(starts,ends)), axis=1))
avg_times = [datetime.utcfromtimestamp(a) for a in avg_times]
avg_times = [dt.isoformat() for dt in avg_times]

In [7]:
masks = ["SCL", "dataMask"] # SCL ... Scene Classification Layer

output_bands = input_bands + indices
output_array =  ','.join([f"{{id: '{ob}', bands: {len(avg_times)}, "+
                          f"sampleType: SampleType.UINT16}}" for ob in output_bands])
int_bands = '{' + ','.join([f'{ib}: []' for ib in input_bands]) + '}'
results_object = '{' + ','.join([f'{ob}: []' for ob in output_bands]) + '}'
debug_results = '{' + ','.join([f"{output_bands[i]}: [{i+1}]" for i in range(len(output_bands))]) + '}'
responses = [{"identifier": ob,"format": {"type": "image/tiff"}} for ob in output_bands]

### Evalscript & Payload

In [8]:
#double curly brackets render as single curly brackets in f-strings
evalscript = f"""
//VERSION=3

var debug = []

var ic = {{  // index components
  'NDVI':  ["B08", "B04"],
  "GNDVI": ["B08", "B03"],
  "BNDVI": ["B08", "B02"],
  "NDSI":  ["B11", "B12"],
  "NDWI":  ["B03", "B08"]
}}

function setup(ds) {{
  return {{
    input: [{{
      bands: {str(input_bands + masks)}, 
      units: "DN"
    }}],
    output: [        
      {output_array}
    ],
    mosaicking: Mosaicking.ORBIT       
  }}
}}

function validate (sample) {{
  if (sample.dataMask!=1) return false;
  
  var scl = Math.round(sample.SCL);
  
  if (scl === 3) {{ // SC_CLOUD_SHADOW
    return false;
  }} else if (scl === 9) {{ // SC_CLOUD_HIGH_PROBA
    return false; 
  }} else if (scl === 8) {{ // SC_CLOUD_MEDIUM_PROBA
    return false;
  }} else if (scl === 7) {{ // SC_CLOUD_LOW_PROBA
    //return false;
  }} else if (scl === 10) {{ // SC_THIN_CIRRUS
    return false;
  }} else if (scl === 11) {{ // SC_SNOW_ICE
    return false;
  }} else if (scl === 1) {{ // SC_SATURATED_DEFECTIVE
    return false;
  }} else if (scl === 2) {{ // SC_DARK_FEATURE_SHADOW
    //return false;
  }}
  return true;
}}

function calculateIndex(a,b)
{{
  if ((a+b)==0) return 0;
  // stretch [-1,+1] to [0,1]
  return ((a-b)/(a+b)+1)/2;
}}

function interpolatedValue(arr)
{{
  //here we define the function on how to define the proper value - e.g. linear interpolation; we will use average 
  if (arr.length==0) return 0;
  if (arr.length==1) return arr[0];
  var sum = 0;
  for (j=0;j<arr.length;j++)
  {{sum+=arr[j];}}
  return Math.round(sum/arr.length);
}}

var results = {results_object}

// We split each month into two halves. This will make it easier to append months to data cube later
var day_of_new_interval = {day_of_new_interval}
var endtime = new Date({datetime(*enddate.timetuple()[:3],23,59,59).timestamp()*1000}) // UNIX epoch in ms

function evaluatePixel(samples, scenes, inputMetadata, customData, outputMetadata) {{
  
  //Debug part returning "something" if there are no  valid samples (no observations)
  if (!samples.length)
  return {debug_results}
  
  var is_in_last_half_of_month = endtime.getUTCDate() >= day_of_new_interval
  var i = 0; // interval number
  var int_bands = {int_bands}
  
  for (var j = 0; j < samples.length; j++) {{
    
    //TODO order should be reversed when we go leastRecent
    
    // if scene is outside of current half of month, fill result array and change half of month
    // algorithm starts with most recent observation
    if (( !is_in_last_half_of_month && scenes[j].date.getUTCDate() >= day_of_new_interval) ||
    (  is_in_last_half_of_month && scenes[j].date.getUTCDate() <  day_of_new_interval))
    {{
      fillResultArray(i, int_bands)
      
      //reset values
      for (var int_b in int_bands) {{
        int_bands[int_b] = []
      }}
      
      is_in_last_half_of_month = !is_in_last_half_of_month;
      i++;
    }}
    
    if (validate(samples[j]))
    {{
      // push input samples into their respective arrays
      for (var int_b in int_bands) {{
        int_bands[int_b].push(samples[j][int_b])
      }}
    }}
    
  }}
  
  //execute this for the last interval 
  fillResultArray(i, int_bands);
  
  return results
}}

function fillResultArray(i, int_bands)
{{
  for (var b in int_bands) {{
    if(int_bands[b].length==0) results[b][i] = 0
    else results[b][i] = interpolatedValue(int_bands[b])
  }}
  
  for (var ix of {indices}) {{
    if(ic.hasOwnProperty(ix)) {{
      results[ix][i] = 65535*calculateIndex(
        results[ic[ix][0]][i],
        results[ic[ix][1]][i]
      )
    }}
    if(ix==="CVI"){{
      results[ix][i] = 65535*results["B08"][i]*results["B05"][i] / (results["B03"][i]*results["B03"][i])
    }}
  }}
}}

function updateOutputMetadata(scenes, inputMetadata, outputMetadata) {{
  outputMetadata.userData = {{
    "date_created": Date(),
    "metadata": scenes.map(s => {{
      s.date = s.date.toString()
      return s
    }}),
    "time" : {avg_times},
    "debug": debug
  }}
}}
"""

In [9]:
payload = {
  "processRequest": {
    "input": {
      "bounds": {
        "properties": {
          "crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"
        },
        "bbox": [16.446445736463346, 47.680841561177864, 16.49776618971013, 47.72587417451863]
      },
      "data": [
        {
          "location": "AWS:eu-central-1",
          "type": "S2L2A",
          "dataFilter": {
            "timeRange": {
              "from": starttime.isoformat() + 'Z',
              "to": endtime.isoformat() + 'Z'
            },
            "mosaickingOrder": "mostRecent",
            "maxCloudCoverage": 100,
            "previewMode": "DETAIL"
          }
        }
      ]
    },
    "output": {
#       "width": 512,
#       "height": 512,
      "responses": [*responses#,
#         {
#           "identifier": "userdata",
#           "format": {
#             "type": "application/json"
#           }
#         }
      ]
    },
    "evalscript": evalscript
  },
  "tilingGridId": 0,
  "bucketName": bucket_name,
  "resolution": 60.0,
  "description": "Test Loipersbach"
}

headers = {
  #'Accept': 'application/tar'
}

## Send request

In [10]:
def generate_url(request_id="", action=""):
    url = 'https://services.sentinel-hub.com/batch/v1/process/'
    if request_id:
        url += f'{request_id}/'
        if action:
            url += f'{action}'
    return url

In [11]:
%%time
response = oauth.request("POST", generate_url(), headers=headers, json = payload)
request_id = response.json()["id"]
response.status_code, response.json()['id'], response.json()['status'], response.json()['valueEstimate']

CPU times: user 4.3 ms, sys: 0 ns, total: 4.3 ms
Wall time: 1.33 s


(201, 'ae9a808d-84d9-4b6b-ba78-3e7856d3a1b6', 'CREATED', 3.6462630964959843)

In [12]:
analysis_response = oauth.request("POST", generate_url(request_id,'analyse'))
analysis_response

<Response [204]>

In [13]:
oauth.request("POST", generate_url(request_id, 'start'))

<Response [204]>

In [14]:
%%time

response_status = ""
while response_status not in ['DONE', 'FAILED']:
    get_response = oauth.request("GET", generate_url(request_id))
    response_status = get_response.json()['status']
    time.sleep(1)

response_status

CPU times: user 375 ms, sys: 19 ms, total: 394 ms
Wall time: 2min 9s


'DONE'

In [15]:
oauth.request("GET", generate_url(request_id)).json()['status']

'DONE'

In [16]:
get_response.status_code, get_response.json()['id'], get_response.json()['status'], get_response.json()['valueEstimate']

(200, 'ae9a808d-84d9-4b6b-ba78-3e7856d3a1b6', 'DONE', 7.281674013077767)

In [17]:
tiles = oauth.request('GET', generate_url(request_id, 'tiles')).json()['member']
[(t['id'], t['cost']) for t in tiles]

[(1621, 7.801793585440464)]

In [18]:
s3 = boto3.resource('s3')
s3.Bucket(bucket_name).put_object(Key=request_id + '/userdata.json', Body=json.dumps({
    'bands': output_bands,
    'request_id': request_id,
    'tiles': tiles,
    'time': avg_times    
}))

s3.Object(bucket_name='eox-masterdatacube', key='ae9a808d-84d9-4b6b-ba78-3e7856d3a1b6/userdata.json')

In [19]:
%store bucket_name
%store request_id
%store

Stored 'bucket_name' (str)
Stored 'request_id' (str)
Stored variables and their in-db values:
bucket_name             -> 'eox-masterdatacube'
request_id              -> 'ae9a808d-84d9-4b6b-ba78-3e7856d3a1b6'
